<a href="https://colab.research.google.com/github/nandodeomkar/BFD-using-ResNet50/blob/main/Full_Fledged_Try_at_BFD_using_ResNet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install all required Libraries

**Libraries** -
Tensorflow,
Keras,
OpenCV,
NumPy,
MatPlotLib,
Scikit-Learn.

In [1]:
!pip install tensorflow
!pip install keras
!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Mount Google Drive and Import Folder as Dataset

#Processing Data 

Modify Names of Files in Training Dataset

In [3]:
import os

# Set the directory path
directory = '/content/drive/MyDrive/Dataset/testing/fractured'

# Set the prefix
prefix = 'Y_'

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Add the prefix to the filename
        new_filename = prefix + filename
        # Rename the file
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))


Resizing

In [ ]:
import os
import cv2

# Set the directory path
directory = '/content/Testing/'

# Set the image size
img_size = (224, 224)

# Loop through all subdirectories in the directory
for subdir, dirs, files in os.walk(directory):
    for filename in files:
        # Check if the file is an image
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Load the image
            img = cv2.imread(os.path.join(subdir, filename))
            # Resize the image
            resized_img = cv2.resize(img, img_size)
            # Save the resized image
            cv2.imwrite(os.path.join(subdir, filename), resized_img)


Normalization

In [ ]:
import cv2
import numpy as np
import os

# Set the directory path
directory = '/content/Training/not_fractured/'

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        img = cv2.imread(os.path.join(directory, filename))
        # Convert the pixel values to float
        img = img.astype('float32')
        # Normalize the pixel values to [0, 1]
        img /= 255.0
        # Save the normalized image
        cv2.imwrite(os.path.join(directory, filename), img)


.jpeg to NumPy Array

In [ ]:
import cv2
import numpy as np
import os

# Set the directory path
directory = '/content/Training/'

# Set the image size
img_size = (224, 224)

# Initialize an empty list to store the images and labels
images = []
labels = []

# Loop through all subdirectories in the directory
for subdir, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Load the image
            img = cv2.imread(os.path.join(subdir, filename))
            # Resize the image
            img = cv2.resize(img, img_size)
            # Add the image to the list
            images.append(img)
            # Extract the label from the filename
            label = subdir.split('/')[-1]
            # Add the label to the list
            labels.append(label)

# Convert the list of images and labels to NumPy arrays
X = np.array(images)
y = np.array(labels)


Import ResNet 50 

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50

# Load the ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# Add a new classification layer
x = resnet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

# Create a new model with the new classification layer
model = tf.keras.models.Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import numpy as np

# Save the X and y arrays to disk
np.save('images.npy', X)
np.save('labels.npy', y)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Define the data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    shear_range=0.2,  # Apply random shear transformations
    zoom_range=0.2,  # Apply random zoom transformations
    horizontal_flip=True  # Flip images horizontally
)

# Define the data generator for validation data
val_datagen = ImageDataGenerator(
    rescale=1./255  # Rescale pixel values to [0, 1]
)

# Load the training data
train_data = train_datagen.flow_from_directory(
    '/content/Training',  # Directory containing the training images
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=32,  # Number of images per batch
    class_mode='categorical'  # Use categorical cross-entropy loss
)

# Load the validation data
val_data = val_datagen.flow_from_directory(
    '/content/Testing',  # Directory containing the validation images
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=32,  # Number of images per batch
    class_mode='categorical'  # Use categorical cross-entropy loss
)


In [ ]:
!pip install keras


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Define the number of classes
num_classes = 2

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a new fully connected layer for classification
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, epochs=10, validation_data=val_data)
